In [1]:
import os
import yaml
from pathlib import Path
import hydra

import torch
from transformers import AutoTokenizer

import sys
from src.models.docsblip_stage2 import DocsBlip
from src.data.docvqa import generate_dataset, BatchContainer

/home/kusuma/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def init_model(ckpt_path) -> torch.nn.Module:
    config_path = os.path.join(ckpt_path, '.hydra/config.yaml')
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    model = hydra.utils.instantiate(config['model'])
    pathlist = Path(ckpt_path).glob('**/*.ckpt')
    filelist = [str(file) for file in pathlist]
    weight_path = filelist[0]
    ckpt = torch.load(weight_path, weights_only=False)#, map_location='cpu')
    weight = ckpt['state_dict']
    model.load_state_dict(weight, strict=False)
    for n, p in model.named_parameters():
        p.requires_grad = False
    model.eval()
    return model

In [3]:
ckpt_path = 'logs/docsblip_stage2/runs/2025-11-11_22-29-28'
model = init_model(ckpt_path)
model.eval()
encoder_tokenizer = AutoTokenizer.from_pretrained("SCUT-DLVCLab/lilt-roberta-en-base")

BertLMHeadModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.out

In [4]:
dataset = generate_dataset(
    annotation_path="data/spdocvqa_qas/val_v1.0_withQT.json",
    ocr_path="data/ocr",
    subset_size=100,
)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 1591.98it/s]


In [11]:
sample = dataset[10]

In [17]:
sample['id']

'id_txpp0227_10'

In [12]:
encoded_tokens = encoder_tokenizer(sample['words'], boxes=sample['bbox'], return_tensors='pt')

In [13]:
query = sample['question']
query

'Who is ‘presiding’ TRRF GENERAL SESSION (PART 1)?'

In [14]:
inputs = BatchContainer(
    input_ids=encoded_tokens['input_ids'],
    input_attention_mask=encoded_tokens['attention_mask'],
    input_bbox=encoded_tokens['bbox'],
    targets=sample['answer'],
    questions=sample['question'],
    words=sample['words'],
)

In [15]:
model.generate_answer(inputs)

['THE GAME.\n\n\n\n\n\n\n\n\n\n\n\n']

In [26]:
sample = dataset[12]
inputs = BatchContainer(
    input_ids=encoded_tokens['input_ids'],
    input_attention_mask=encoded_tokens['attention_mask'],
    input_bbox=encoded_tokens['bbox'],
    targets=sample['answer'],
    questions=sample['question'],
    words=sample['words'],
)
question = sample['question']
answer = model.generate_answer(inputs)[0]
dialog = f"""User: {question}\nAssistant: {answer}"""
print(dialog)
print(f"GT: {sample['answer']}")

User: What is the name of the fashion wear/clothing advertise
Assistant: THE GAME.
the game

these as this page


GT: wills lifestyle


In [25]:
sample = dataset[33]
inputs = BatchContainer(
    input_ids=encoded_tokens['input_ids'],
    input_attention_mask=encoded_tokens['attention_mask'],
    input_bbox=encoded_tokens['bbox'],
    targets=sample['answer'],
    questions=sample['question'],
    words=sample['words'],
)
question = sample['question']
answer = model.generate_answer(inputs)[0]
dialog = f"""User: {question}\nAssistant: {answer}"""
print(dialog)
print(f"GT: {sample['answer']}")


User: How many lint/string pieces of matter is found in the core samples ?
Assistant: the game
Ё by the game.
by the rest of this game

GT: 22
